# Imports

In [2]:
import os
import time
import torch
import chess
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, random_split

# Paths

In [3]:
base_dir = "/home/mystic/Programming/mystic-bot/notebooks"
data_path = f"{base_dir}/data/chessData.csv"
model_dir = f"{base_dir}/models"
plot_dir = f"{base_dir}/plots"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)
GLOBAL_DATA = pd.read_csv(data_path, usecols=["FEN", "Evaluation"])

# Device

In [4]:
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"✅ {device_count} CUDA device(s) available:")
    for i in range(device_count):
        print(f"  └─ [{i}] {torch.cuda.get_device_name(i)}")
    device = torch.device("cuda")
else:
    print("⚠️ CUDA not available, using CPU.")
    device = torch.device("cpu")

✅ 1 CUDA device(s) available:
  └─ [0] NVIDIA GeForce GTX 1650


# Pre-Process Data

In [5]:
def fen_to_tensor(fen: str) -> np.ndarray:
    piece_map = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }

    tensor = np.zeros((18, 8, 8), dtype=np.float32)
    board = chess.Board(fen)

    # --- Piece Planes (12 channels) ---
    for square, piece in board.piece_map().items():
        row = 7 - (square // 8)  # White at bottom
        col = square % 8
        tensor[piece_map[piece.symbol()], row, col] = 1.0

    # --- Turn (White to move: 1) [1 cell] ---
    tensor[12, 0, 0] = 1.0 if board.turn == chess.WHITE else 0.0

    # --- En Passant Square [1 cell] ---
    if board.ep_square is not None:
        row = 7 - (board.ep_square // 8)
        col = board.ep_square % 8
        tensor[13, row, col] = 1.0

    # --- Castling Rights [1 cell each] ---
    tensor[14, 0, 0] = float(
        board.has_kingside_castling_rights(chess.WHITE))  # WK
    tensor[15, 0, 0] = float(
        board.has_queenside_castling_rights(chess.WHITE))  # WQ
    tensor[16, 0, 0] = float(
        board.has_kingside_castling_rights(chess.BLACK))  # BK
    tensor[17, 0, 0] = float(
        board.has_queenside_castling_rights(chess.BLACK))  # BQ

    return tensor

# Dataset Encoder

In [6]:
def parse_eval(val):
    try:
        return float(val)
    except ValueError:
        return np.nan


class ChessPositionDataset(Dataset):
    def __init__(self, max_samples=None):
        df = GLOBAL_DATA.copy()
        df = df.dropna(subset=["FEN", "Evaluation"])

        # Parse and normalize evaluation values
        df["Evaluation"] = df["Evaluation"].apply(parse_eval)
        df = df.dropna(subset=["Evaluation"])

        # Shuffle and trim
        if max_samples:
            df = df.sample(frac=1, random_state=42).iloc[:max_samples]

        self.max_score = df["Evaluation"].abs().max()
        self.samples = df.to_dict("records")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        # Should be (18, 8, 8) np array
        input_tensor = fen_to_tensor(sample["FEN"])
        input_tensor = torch.tensor(input_tensor, dtype=torch.float32)

        eval_score = float(sample["Evaluation"]) / self.max_score
        eval_score = torch.tensor(eval_score, dtype=torch.float32)

        return input_tensor, eval_score

# Loaders

In [7]:
# --- Load full dataset ---
dataset = ChessPositionDataset(
    max_samples=200_000  # You can change this based on memory/batch size
)

# --- Split sizes ---
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# --- Split dataset ---
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # For reproducibility
)

# --- Create DataLoaders ---
BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model

In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(channels)
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(x + self.block(x))


class EvalNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(18, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            ResidualBlock(64),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            ResidualBlock(128),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            ResidualBlock(256)
        )

        # Global average pooling instead of full flatten
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.gap(x)
        return self.head(x)


model = EvalNet().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.SmoothL1Loss()
summary(model, input_size=(18, 8, 8), device=str(device))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 8, 8]          10,432
       BatchNorm2d-2             [-1, 64, 8, 8]             128
              ReLU-3             [-1, 64, 8, 8]               0
            Conv2d-4             [-1, 64, 8, 8]          36,928
       BatchNorm2d-5             [-1, 64, 8, 8]             128
              ReLU-6             [-1, 64, 8, 8]               0
            Conv2d-7             [-1, 64, 8, 8]          36,928
       BatchNorm2d-8             [-1, 64, 8, 8]             128
              ReLU-9             [-1, 64, 8, 8]               0
    ResidualBlock-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 128, 8, 8]          73,856
      BatchNorm2d-12            [-1, 128, 8, 8]             256
             ReLU-13            [-1, 128, 8, 8]               0
           Conv2d-14            [-1, 12

In [9]:
# Initial Validation (Untrained Model)
model.eval()
total_val_loss = 0.0
val_steps = 0

print("\n🔎 Initial Validation (Untrained Model)...")
val_pbar = tqdm(val_loader, desc="Validation",
                unit="batch", total=len(val_loader))
with torch.no_grad():
    for x, y in val_pbar:
        x, y = x.to(device), y.to(device)
        pred = model(x).squeeze()
        loss = loss_fn(pred, y.squeeze())

        total_val_loss += loss.item()
        val_steps += 1
        val_pbar.set_postfix({"val_loss": f"{loss.item():.4f}"})

initial_val_loss = total_val_loss / val_steps
print(f"\n🧪 Initial Val Loss (Untrained): {initial_val_loss:.4f}")


🔎 Initial Validation (Untrained Model)...


Validation:   0%|          | 0/313 [00:00<?, ?batch/s]


🧪 Initial Val Loss (Untrained): 0.0047


# Training Loop

In [10]:
train_losses, val_losses = [], []
best_val_loss = float("inf")
epochs = 50

# Training Loop
for epoch in range(epochs):
    model.train()
    total_train_loss = 0.0
    train_steps = 0

    print(f"\n📚 Epoch {epoch+1}/{epochs} - Training...")
    pbar = tqdm(train_loader, desc="Training",
                unit="batch", total=len(train_loader))
    for x, y in pbar:
        x, y = x.to(device), y.to(device)
        pred = model(x).squeeze()
        loss = loss_fn(pred, y.squeeze())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        train_steps += 1
        pbar.set_postfix({"batch_loss": f"{loss.item():.4f}"})

    avg_train_loss = total_train_loss / train_steps
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    total_val_loss = 0.0
    val_steps = 0

    print(f"\n🔎 Epoch {epoch+1}/{epochs} - Validation...")
    val_pbar = tqdm(val_loader, desc="Validation",
                    unit="batch", total=len(val_loader))
    with torch.no_grad():
        for x, y in val_pbar:
            x, y = x.to(device), y.to(device)
            pred = model(x).squeeze()
            loss = loss_fn(pred, y.squeeze())

            total_val_loss += loss.item()
            val_steps += 1
            val_pbar.set_postfix({"val_loss": f"{loss.item():.4f}"})

    avg_val_loss = total_val_loss / val_steps
    val_losses.append(avg_val_loss)

    # Logging
    print(
        f"\n✅ Epoch {epoch+1} Summary | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Checkpointing
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    torch.save(model.state_dict(), os.path.join(model_dir, f"{timestamp}.pt"))
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), os.path.join(model_dir, "best.pt"))

    # Plotting (no display)
    plt.figure(figsize=(12, 6), dpi=150)
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training & Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, "loss_curve.png"))
    plt.close()

# Final Evaluation
print("\n🔍 Loading best model for final test...")
model.load_state_dict(torch.load(os.path.join(model_dir, "best.pt")))
model.eval()


📚 Epoch 1/50 - Training...


Training:   0%|          | 0/2500 [00:00<?, ?batch/s]


🔎 Epoch 1/50 - Validation...


Validation:   0%|          | 0/313 [00:00<?, ?batch/s]


✅ Epoch 1 Summary | Train Loss: 0.0040 | Val Loss: 0.0037

📚 Epoch 2/50 - Training...


...


🔎 Epoch 10/50 - Validation...


Validation:   0%|          | 0/313 [00:00<?, ?batch/s]


✅ Epoch 10 Summary | Train Loss: 0.0009 | Val Loss: 0.0037

📚 Epoch 11/50 - Training...


Training:   0%|          | 0/2500 [00:00<?, ?batch/s]

KeyboardInterrupt: 

# Evaluation

In [ ]:
total_test_loss = 0.0
test_steps = 0

print("\n🧪 Evaluating on test set...")
test_pbar = tqdm(test_loader, desc="Testing",
                 unit="batch", total=len(test_loader))
with torch.no_grad():
    for x, y in test_pbar:
        x, y = x.to(device), y.to(device)
        pred = model(x).squeeze()
        loss = loss_fn(pred, y.squeeze())

        total_test_loss += loss.item()
        test_steps += 1
        test_pbar.set_postfix({"batch_loss": f"{loss.item():.4f}"})

avg_test_loss = total_test_loss / test_steps
print(f"\n🎯 Final Test Loss: {avg_test_loss:.4f}")


🧪 Evaluating on test set...


Testing:   0%|          | 0/313 [00:00<?, ?batch/s]


🎯 Final Test Loss: 0.0072


# Manual

In [ ]:
def evaluate_fen(model, fen):
    model.eval()

    with torch.no_grad():
        tensor = torch.tensor(fen_to_tensor(
            fen), dtype=torch.float32).unsqueeze(0).to(device)
        pred = model(tensor).squeeze().item()

    return pred


fen = "8/8/8/P7/4pk2/2R3p1/5PK1/4B3 b - - 0 44"
score = evaluate_fen(model, fen)*100
print(f"Predicted Evaluation: {score:.4f}")

Predicted Evaluation: 0.3852
